   #  ---------------XGBoost-Survival-SHAP--------------

In [ ]:
import shap
import xgboost
from sklearn.model_selection import train_test_split
import matplotlib.pylab as pl
import warnings
from pandas import DataFrame
warnings.filterwarnings(action='ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
zh = DataFrame(np.arange(1).reshape(1,1))

dataDir = r'E:\My_project_NPC\XGB-Cox'
train_filename = '/T1_T2_risk_train.csv'
test_filename = '/T1_T2_risk_test.csv'
val_filename = '/T1_T2_risk_val.csv'
train_data = pd.read_csv(dataDir + train_filename)

In [ ]:
cc = 3
y_train = (np.where(train_data.iloc[:, 1] == 0, -1, train_data.iloc[:, 1])) * train_data.iloc[:, 2]
x_train = train_data.iloc[:,cc:]

## --------------------internal validation----------------------
test_data = pd.read_csv(dataDir + test_filename)
x_test = test_data.iloc[:,cc:]
y_test = (np.where(test_data.iloc[:, 1] == 0, -1, test_data.iloc[:, 1])) * test_data.iloc[:, 2]

## --------------------external validation----------------------
val_data = pd.read_csv(dataDir + val_filename)
x_val = val_data.iloc[:,cc:]
y_val = (np.where(val_data.iloc[:, 1] == 0, -1, val_data.iloc[:, 1])) * val_data.iloc[:, 2]

In [ ]:
# --------------- model save ----------------------
import time
format_date = time.strftime("Model_%Y-%m-%d_%H.%M", time.localtime(time.time()))
import os
os.mkdir(dataDir + '/' + format_date)

In [ ]:
import xgboost
train = xgboost.DMatrix(x_train,y_train)
test = xgboost.DMatrix(x_test,y_test)
dtrain = xgboost.DMatrix(x_train)
dtest = xgboost.DMatrix(x_test)
dval = xgboost.DMatrix(x_val)

In [ ]:
from lifelines.utils import concordance_index
best_score = 0
score_list = []
print('Baseline:', best_score)
for seed in [1014]:  ## optimal hyperparameters
    for n_estimators in [7]: 
        for max_depth in [3]:
            for colsample_bytree in [0.5]:
                for subsample in [0.3]:
                    for child in [2]:
                        for eta in [0.3]:
                            for gamma in [1]:
                                for lambda1 in [0.00005]:
                                    for alpha in [0.001]:
                                        for min_child_weight in [2]:
                                            model_forest = xgboost.train(
                                                {'objective': 'survival:cox', 'max_depth': max_depth, 'colsample_bytree': colsample_bytree,
                                                 'min_child_weight': min_child_weight, 'subsample': subsample, 'booster': 'gbtree', 'eta': eta,
                                                 'gamma':gamma, 'lambda':lambda1,'alpha':alpha,'min_child_weight':min_child_weight,
                                                 'random_state':seed}, train, n_estimators)
                                            train_score = round(concordance_index(train_data['futime'], -model_forest.predict(dtrain), train_data['fustat']), 3)
                                            test_score = round(concordance_index( test_data['futime'], -model_forest.predict( dtest),  test_data['fustat']), 3)
                                            val_score = round(concordance_index(  val_data['futime'], -model_forest.predict(  dval),   val_data['fustat']), 3)
                                            score = min(round(train_score,3), round(test_score,3), round(val_score,3))
                                            
                                            if score > best_score:
                                                best_score = score
                                                best_parameters = {'seed': seed, 'estimators': n_estimators, 'depth': max_depth,'gamma':gamma,'alpha':alpha,'min_child_weight':min_child_weight,
                                                                   'lambda':lambda1,'colsample': colsample_bytree, 'subsample': subsample, 
                                                                   'eta':eta}
                                                best_model = model_forest
                                                print('\nBest_score', round(best_score, 3), train_score, test_score, val_score)
                                                print(best_parameters)
print('\ finish')


In [ ]:
# ----------------------------forest model save-------------------------------------
import joblib
print(dataDir + '\\\\' + format_date + '\\\\' + '  model_Xgboost.pkl')
joblib.dump(best_model, dataDir + '\\\\' + format_date + '\\\\' +'Model_Xgboost.pkl')

In [ ]:
train_score = concordance_index(train_data['futime'], -best_model.predict(dtrain), train_data['fustat'])
test_score = concordance_index(test_data['futime'], -best_model.predict(dtest), test_data['fustat'])
val_score = concordance_index(val_data['futime'], -best_model.predict(dval), val_data['fustat'])
print('\ntrain_score',round(train_score,3),'\ntest_score', round(test_score,3),'\nval_score', round(val_score,3))

train_data['riskScore'] = best_model.predict(dtrain)
test_data['riskScore'] = best_model.predict(dtest)
val_data['riskScore'] = best_model.predict(dval)
train_data.to_csv(dataDir + '/' + format_date + '/risk-train.csv')
test_data.to_csv(dataDir + '/' + format_date + '/risk-test.csv')
val_data.to_csv(dataDir + '/' + format_date + '/risk-val.csv')

## ---------------------SHAP ---------------------------

In [ ]:
import shap
shap.initjs()
explainer = shap.TreeExplainer(best_model)
shap_values = explainer(x_train)

In [ ]:
#-------------------summary plot--------------------
Max_display = 10

myfig = plt.gcf()
sum_shap_values = explainer.shap_values(x_train)
shap.summary_plot(sum_shap_values, x_train, max_display=Max_display) #, plot_type="violin"
myfig.savefig(dataDir + '\\' + format_date + '\\' + '2. Beeswarm_train.pdf',dpi=300,bbox_inches = 'tight')

myfig = plt.gcf()
sum_shap_values_test = explainer.shap_values(x_test)
shap.summary_plot(sum_shap_values_test, x_test, max_display=Max_display) # , plot_type="violin"
myfig.savefig(dataDir + '\\' + format_date + '\\' + '2. Beeswarm_test.pdf',dpi=300,bbox_inches = 'tight')

myfig = plt.gcf()
sum_shap_values_val = explainer.shap_values(x_val)
shap.summary_plot(sum_shap_values_val, x_val, max_display=Max_display) # , plot_type="violin"
myfig.savefig(dataDir + '\\' + format_date + '\\' + '3. Beeswarm_val.pdf',dpi=300,bbox_inches = 'tight')

In [ ]:
for column in x_train.columns:
    shap.plots.scatter(shap_values[:, column], color=shap_values[:,column],show=False)
    myfig = plt.gcf()
    shap.plots.scatter(shap_values[:, column], color=shap_values[:,column],show=False)
    myfig.savefig(dataDir + '\\' + format_date + '\\' + column +'.tiff',dpi=300)
    plt.clf()

In [ ]:
#--------------force plot-----------------
for i in range(0, 30, 1):
    plt.clf()
    shap.initjs()
    shap.plots.waterfall(shap_values[i], show=False)
    myfig = plt.gcf()
    print("id:", i + 1)
    myfig.savefig(dataDir + '\\' + format_date + '\\' + str(i+1)+'.tif',dpi=300,bbox_inches = 'tight')
    myfig.savefig(dataDir + '\\' + format_date + '\\' + str(i+1)+'.pdf',dpi=300,bbox_inches = 'tight')